In [1]:
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [12]:
env = GridworldEnv()

In [16]:
env.nS

16

In [57]:
env.P[0]

{0: [(1.0, 0, 0.0, True)],
 1: [(1.0, 0, 0.0, True)],
 2: [(1.0, 0, 0.0, True)],
 3: [(1.0, 0, 0.0, True)]}

In [32]:
env.nS

16

In [46]:
random_policy

array([[0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25]])

In [47]:
policy = random_policy
R_pi = [ np.sum([ policy[s][a] * env.P[s][a][0][2] for a in range(4)])
        for s in range(env.nS)]

In [48]:
len(R_pi)

16

In [49]:
R_pi

[0.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 0.0]

In [66]:
R_pi = np.array([np.sum([policy[s][a] * env.P[s][a][0][2]
                         for a in range(env.nA)])
                 for s in range(env.nS)])

P_pi = np.array([
                    [
                         np.sum([policy[s_from][a]
                         for a in range(env.nA)
                         if env.P[s_from][a][0][1]==s_to])
                         for s_to in range(env.nS)]
                 for s_from in range(env.nS)])



In [68]:
R_pi = R_pi.reshape((-1,1))

In [69]:
R_pi

array([[ 0.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [ 0.]])

In [67]:
P_pi

array([[1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.25, 0.25, 0.25, 0.  , 0.  , 0.25, 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.25, 0.25, 0.25, 0.  , 0.  , 0.25, 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.25, 0.5 , 0.  , 0.  , 0.  , 0.25, 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.25, 0.  , 0.  , 0.  , 0.25, 0.25, 0.  , 0.  , 0.25, 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.25, 0.  , 0.  , 0.25, 0.  , 0.25, 0.  , 0.  , 0.25, 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.25, 0.  , 0.  , 0.25, 0.  , 0.25, 0.  , 0.  , 0.25,
        0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.25, 0.  , 0.  , 0.25, 0.25, 0.  , 0.  , 0.  ,
        0.25, 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.25, 0.  , 0.  , 0.  , 0.25, 0.25, 0.  ,
        0.  , 0.25, 0.

In [152]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    V = V.reshape((-1,1))
    
    R_pi = np.array([np.sum([policy[s][a] * env.P[s][a][0][2]
                             for a in range(env.nA)])
                     for s in range(env.nS)])
    R_pi = R_pi.reshape((-1,1))

    P_pi = np.array([
                        [
                             np.sum([policy[s_from][a]
                             for a in range(env.nA)
                             if env.P[s_from][a][0][1]==s_to])
                             
                         for s_to in range(env.nS)]
                     for s_from in range(env.nS)])

    while True:
        # TODO: Implement!
        V_new = R_pi + discount_factor * np.matmul(P_pi, V)
        if np.max(np.abs(V_new - V)) < theta:
            V = V_new
            break
        V = V_new
    return V.flatten().tolist()

In [153]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)

In [154]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)